In [19]:
import sys
!{sys.executable} -m pip install -r requirements.txt

Import all the stuff we just installed from Cell one 

In [20]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper
import project_tests
import bs4
import lxml
import html5lib
import bs4
import urllib3
import requests

from tqdm import tqdm

Now we add two corpora from nltk package stopwords removes stopwaords and wordnet lemmatizes

In [21]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mrhea\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mrhea\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Get 10Ks

To get all the 10ks for this project we'll use their CIKs to lookup the documents from the SEC. We will load them in the next code 

In [22]:
cik_lookup = {
    'AMZN': '0001018724',
    'BMY': '0000014272',   
    'CNP': '0001130310',
    'CVX': '0000093410',
    'FL': '0000850209',
    'FRT': '0000034903',
    'HON': '0000773840'}

additional_cik = {
    'AEP': '0000004904',
    'AXP': '0000004962',
    'BA': '0000012927', 
    'BK': '0001390777',
    'CAT': '0000018230',
    'DE': '0000315189',
    'DIS': '0001001039',
    'DTE': '0000936340',
    'ED': '0001047862',
    'EMR': '0000032604',
    'ETN': '0001551182',
    'GE': '0000040545',
    'IBM': '0000051143',
    'IP': '0000051434',
    'JNJ': '0000200406',
    'KO': '0000021344',
    'LLY': '0000059478',
    'MCD': '0000063908',
    'MO': '0000764180',
    'MRK': '0000310158',
    'MRO': '0000101778',
    'PCG': '0001004980',
    'PEP': '0000077476',
    'PFE': '0000078003',
    'PG': '0000080424',
    'PNR': '0000077360',
    'SYY': '0000096021',
    'TXN': '0000097476',
    'UTX': '0000101829',
    'WFC': '0000072971',
    'WMT': '0000104169',
    'WY': '0000106535',
    'XOM': '0000034088'}

print(cik_lookup)

{'AMZN': '0001018724', 'BMY': '0000014272', 'CNP': '0001130310', 'CVX': '0000093410', 'FL': '0000850209', 'FRT': '0000034903', 'HON': '0000773840'}


Now that these are loaded we will pull up classes from our helper document that helps cache SEC data and avoid going over the SEC call limits.

In [23]:
sec_api = project_helper.SecAPI()

In [38]:
from bs4 import BeautifulSoup

def get_sec_data(cik, doc_type):
    newest_pricing_data = pd.to_datetime('2023-01-01')
    rss_url = 'https://www.sec.gov/Archives/edgar/data/1122304/0001193125-15-118890.txt' \
        .format(cik, doc_type)
    headers = {'User-Agent': 'Mozilla/5.0'}
    sec_data = requests.get(rss_url, headers=headers).text
    feed = BeautifulSoup(sec_data.encode('ascii'), 'xml').feed
    entries = [
        (
            entry.content.find('filing-href').getText(),
            entry.content.find('filing-type').getText(),
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)
        if pd.to_datetime(entry.content.find('filing-date').getText()) <= newest_pricing_data]

    return entries


Now lets take an example ticker from Amazon or AMZN and check that our links work

In [39]:
example_ticker = 'AMZN'
sec_data = {}

print(sec_data)
for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik, '10-K')

pprint.pprint(sec_data[example_ticker][:5])

{}


NameError: name 'doc_type' is not defined